In [ ]:
# Script that receives an unclassified PSF dataset and returns the dataset with the according SEDs

import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA

In [ ]:
#PCA decomposition
dataset_path = '/Users/as274094/Documents/psf_dataset2/'
test_dataset_name = 'no_SED_test_Euclid_res_20000_TestStars_id_002GT_100_bins.npy'
test_dataset = np.load(dataset_path + test_dataset_name, allow_pickle=True)[()]
train_dataset = np.load(dataset_path + 'train_Euclid_res_52000_TrainStars_id_002GT_100_bins.npy', allow_pickle=True)[()]

# Load the stars
noisy_train_stars = train_dataset['noisy_stars']
noisy_test_stars = test_dataset['noisy_stars']

#should I include the validation stars as well?

fit_selection = np.concatenate((noisy_train_stars, noisy_test_stars), axis = 0)
N_components = 30

pca = PCA(n_components= N_components)
pca.fit(fit_selection.reshape(-1, 1024))
x_test = pca.transform(noisy_test_stars.reshape(-1, 1024))

In [ ]:
# Load model and make predictions

model_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/best_models/config1_PCA_dataset2B30/'
classifier = tf.keras.models.load_model(model_path)

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

C_pred = classifier.predict(x_test, verbose = 1).reshape(-1) # Predict the scalar parameter C
class_pred = CtoSEDarray(C_pred,np.zeros_like(C_pred))



In [ ]:
# Assign SEDs
concatenated_SEDs = np.load('concatenated_SEDs.npy', allow_pickle=True)[()]

SED_list = []
for spectral_class in class_pred:
    if spectral_class == 20:
        concat_SED = None # what sould I do about the anomalies?
    else:
        concat_SED = concatenated_SEDs[spectral_class]
    SED_list.append(concat_SED)
SED_array = np.array(SED_list, dtype=object)

In [42]:
test_dataset['SEDs'] = SED_array 
test_dataset['SED_ids'] = class_pred

output_dataset_name = test_dataset_name.replace('no_SED', 'classified')
np.save(
        dataset_path + output_dataset_name,
        test_dataset,
        allow_pickle=True
    )